### 7. 수집한 데이터 기반 사전 만들기

In [1]:
from bs4 import BeautifulSoup
import requests
import re

cine21_url = 'http://www.cine21.com/rank/person/content'
post_data = dict()
post_data['section']='actor'
post_data['period_start']='2020-12'
post_data['gender']='all'
post_data['page']=1
res = requests.post(cine21_url, data=post_data)

soup = BeautifulSoup(res.content, 'html.parser')

In [ ]:
# 4.배우이름 추출
actors = soup.select('li.people_li div.name > a') # li는 리스트를 의미
for actor in actors:
    print(re.sub('\(\w*\)', '', actor.text)) # \( \w * \)

In [ ]:
# 5.배우 상세 정보 추출
actors = soup.select('li.people_li div.name > a')

actors_info_list = list()

for actor in actors:
    actor_link = 'http://www.cine21.com' + actor.attrs['href'] # 배우 링크
    response_actor = requests.get(actor_link) # 배우 링크 가져오기
    soup_actor = BeautifulSoup(response_actor.content, 'html.parser') # 배우 링크 파싱하기
    default_info = soup_actor.select_one('ul.default_info') # 배우 기본 정보
    actor_details = default_info.select('li') # 기본 정보내에서 상세정보 데이터 추출

    actor_info_dict = dict()
    
    for actor_item in actor_details:
        actor_item_field = actor_item.select_one('span.tit').text # 원하는 값 - 직업
        actor_item_value = re.sub('<span.*?>.*?</span>', '', str(actor_item))
        actor_item_value = re.sub('<.*?>', '', actor_item_value) # 원하는 값 - 배우
        actor_info_dict[actor_item_field] = actor_item_value # 딕셔너리로 저장 'field':'value'
    actors_info_list.append(actor_info_dict)

actors_info_list        

In [ ]:
# 6.배우 흥행 지수와 출연 영화 추출
actors = soup.select('li.people_li div.name > a') # 배우이름
hits = soup.select('ul.num_info > li > strong') # 흥행지수
movies = soup.select('ul.mov_list') # 출연영화 
for index, actor in enumerate(actors):
    print("배우이름:", (re.sub('\(\w*\)', '', actor.text)))
    print("흥행지수:", int(hits[index].text.replace(',','')))
    # 각 배우마다 출연영화가 <li.people_li> 리스트로 나뉘어져 있고  
    # 영화 별로 <ul,mov_list>리스트로 나뉘어져 있음
    movie_titles = movies[index].select('li a span') 
    movie_title_list = list() # <ul,mov_list>의 리스트를 하나의 리스트로 묶기 위함
    for movie_title in movie_titles:
        movie_title_list.append(movie_title.text)
    print("출연영화:", movie_title_list)

In [2]:
actors_info_list = list()

actors = soup.select('li.people_li div.name > a') # 배우이름
hits = soup.select('ul.num_info > li > strong') # 흥행지수
movies = soup.select('ul.mov_list') # 출연영화 
rankings = soup.select('li.people_li > span.grade') # 랭킹

for index, actor in enumerate(actors):
    actor_name = re.sub('\(\w*\)','',actor.text)
    actor_hits = int(hits[index].text.replace(',',''))
    movie_titles = movies[index].select('li a span')
    movie_title_list = list()
    for movie_title in movie_titles:
        movie_title_list.append(movie_title.text)
    actor_info_dict = dict()
    actor_info_dict['배우이름'] = actor_name
    actor_info_dict['흥행지수'] = actor_hits
    actor_info_dict['출연영화'] = movie_title_list
    actor_info_dict['랭킹'] = rankings[index].text
    
    actor_link = 'http://www.cine21.com' + actor.attrs['href'] # 배우 링크
    response_actor = requests.get(actor_link) # 배우 링크 가져오기
    soup_actor = BeautifulSoup(response_actor.content, 'html.parser') # 배우 링크 파싱하기
    default_info = soup_actor.select_one('ul.default_info') # 배우 기본 정보
    actor_details = default_info.select('li') # 기본 정보내에서 상세정보 데이터 추출
    
    for actor_item in actor_details:
        actor_item_field = actor_item.select_one('span.tit').text # 원하는 값
        actor_item_value = re.sub('<span.*?>.*?</span>', '', str(actor_item))
        actor_item_value = re.sub('<.*?>', '', actor_item_value) # 원하는 값
        actor_info_dict[actor_item_field] = actor_item_value # 딕셔너리로 저장 'field':'value'
    actors_info_list.append(actor_info_dict)

actors_info_list
    

[{'배우이름': '김윤석',
  '흥행지수': 45708,
  '출연영화': ['모가디슈', '타짜', '당신, 거기 있어줄래요', '화이 : 괴물을 삼킨 아이', '미성년', '1987'],
  '랭킹': '1',
  '직업': '배우',
  '생년월일': '1968-01-21',
  '성별': '남',
  '신장/체중': '178cm, 68kg',
  '학교': '동의대학교 학사',
  '취미': '여행, 낚시',
  '소속사': '심엔터테인먼트'},
 {'배우이름': '조인성',
  '흥행지수': 39995,
  '출연영화': ['모가디슈', '클래식'],
  '랭킹': '2',
  '다른 이름': '趙寅成',
  '직업': '배우',
  '생년월일': '1981-07-28',
  '성별': '남',
  '신장/체중': '186cm, 72kg',
  '학교': '전남과학대 모델이벤트과 - 동국대 연극영화 (중퇴)',
  '취미': '모자모으기, 영화보기, 농구',
  '특기': '농구, 태권도(공인 4단)',
  '소속사': '싸이더스 HQ'},
 {'배우이름': '허준호',
  '흥행지수': 34251,
  '출연영화': ['모가디슈', '국가부도의 날'],
  '랭킹': '3',
  '원어명': '許俊豪',
  '직업': '배우',
  '생년월일': '1964-03-03',
  '성별': '남',
  '신장/체중': '180cm, 75kg',
  '학교': '서울예술대학 연극과',
  '취미': '만화책 보기',
  '특기': '야구, 농구',
  '소속사': '지티비엔터테인먼트'},
 {'배우이름': '김소진',
  '흥행지수': 28556,
  '출연영화': ['모가디슈', '스프링 송', '아이 캔 스피크', '미성년'],
  '랭킹': '4',
  '직업': '배우',
  '성별': '여'},
 {'배우이름': '조우진',
  '흥행지수': 26118,
  '출연영화': ['발신제한', '도굴', '킹메이커', '서복', '국가부도의 날', 